In [ ]:
!pip3 install ktrain

     |████████████████████████████████| 25.3MB 132kB/s 
     |████████████████████████████████| 6.8MB 50.0MB/s 
     |████████████████████████████████| 983kB 47.4MB/s 
     |████████████████████████████████| 266kB 55.7MB/s 
     |████████████████████████████████| 1.3MB 45.6MB/s 
     |████████████████████████████████| 1.2MB 45.0MB/s 
     |████████████████████████████████| 471kB 49.3MB/s 
     |████████████████████████████████| 2.9MB 39.0MB/s 
     |████████████████████████████████| 890kB 43.0MB/s 
  Created wheel for ktrain: filename=ktrain-0.25.4-cp36-none-any.whl size=25276443 sha256=2dfa979ed87b2b5c6e4e348f522e871f0658af84a97490679f098520f2a6af91
  Stored in directory: /root/.cache/pip/wheels/1b/77/8a/bdceaabc308e7178d575278bf6143b7d1a9b939a1e40c56b88
  Created wheel for langdetect: filename=langdetect-1.0.8-cp36-none-any.whl size=993194 sha256=7b61d61fa7a7913845de3cafd091b5ad7c64c66f71c4d5afaeb876018d69dd61
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d4

In [ ]:
import pandas as pd
import numpy as np

import ktrain
from ktrain import text

In [ ]:
cd drive/MyDrive/bert-project-data/

/content/drive/MyDrive/bert-project-data


In [ ]:
data_train = pd.read_csv('./data_train.csv', encoding='utf-8')
data_test = pd.read_csv('./data_test.csv', encoding='utf-8')


X_train = data_train.Text.tolist()
X_test = data_test.Text.tolist()

y_train = data_train.Emotion.tolist()
y_test = data_test.Emotion.tolist()

data = data_train.append(data_test, ignore_index=True)

class_names = ['joy', 'sadness', 'fear', 'anger', 'neutral']

print('size of training set: %s' % (len(data_train['Text'])))
print('size of validation set: %s' % (len(data_test['Text'])))
print(data.Emotion.value_counts())

data.head(10)

SIZE_TRAIN = 1000
X_train = X_train[:1000]
y_train = y_train[:1000]

size of training set: 7934
size of validation set: 3393
joy        2326
sadness    2317
anger      2259
neutral    2254
fear       2171
Name: Emotion, dtype: int64


In [ ]:
encoding = {
    'joy': 0,
    'sadness': 1,
    'fear': 2,
    'anger': 3,
    'neutral': 4
}

# Integer values for each class

y_train = [encoding[x] for x in y_train]
y_test = [encoding[x] for x in y_test]

In [ ]:
train_t,test_t, preproc = text.texts_from_array(x_train=X_train, y_train=y_train,
                                                                       x_test=X_test, y_test=y_test,
                                                                       class_names=class_names,
                                                                       preprocess_mode='distilbert',
                                                                       maxlen=350, 
                                                                       max_features=35000)

preprocessing train...
language: en
train sequence lengths:
	mean : 17
	95percentile : 41
	99percentile : 55


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 17
	95percentile : 40
	99percentile : 57


task: text classification


In [ ]:
model = text.text_classifier('distilbert', train_data=train_t, preproc=preproc)

Is Multi-Label? False
maxlen is 350



done.


In [ ]:
learner = ktrain.get_learner(model, train_data=train_t, val_data=test_t, batch_size=6)

In [ ]:
learner.fit_onecycle(2e-5, 4)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/4
167/167 [==============================] - 61s 318ms/step - loss: 1.5960 - accuracy: 0.2393 - val_loss: 1.3815 - val_accuracy: 0.4807
Epoch 2/4
167/167 [==============================] - 54s 324ms/step - loss: 1.2387 - accuracy: 0.5773 - val_loss: 0.8943 - val_accuracy: 0.6894
Epoch 3/4
167/167 [==============================] - 55s 329ms/step - loss: 0.5751 - accuracy: 0.8346 - val_loss: 0.7496 - val_accuracy: 0.7409
Epoch 4/4
167/167 [==============================] - 55s 328ms/step - loss: 0.2733 - accuracy: 0.9399 - val_loss: 0.7225 - val_accuracy: 0.7504


In [ ]:
learner.validate(val_data=test_t, class_names=class_names)

              precision    recall  f1-score   support

         joy       0.80      0.79      0.79       707
     sadness       0.70      0.74      0.72       676
        fear       0.83      0.73      0.78       679
       anger       0.72      0.67      0.69       693
     neutral       0.71      0.83      0.77       638

    accuracy                           0.75      3393
   macro avg       0.75      0.75      0.75      3393
weighted avg       0.75      0.75      0.75      3393



array([[555,  36,  18,  11,  87],
       [ 25, 503,  30,  79,  39],
       [ 37,  54, 497,  70,  21],
       [ 31,  91,  43, 464,  64],
       [ 47,  30,  10,  24, 527]])

In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc)
predictor.get_classes()

['joy', 'sadness', 'fear', 'anger', 'neutral']

In [ ]:
import time 

message = 'I just broke up with my boyfriend'

start_time = time.time() 
prediction = predictor.predict(message)

print('predicted: {} ({:.2f})'.format(prediction, (time.time() - start_time)))

predicted: sadness (0.06)


In [ ]:
# let's save the predictor for later use
predictor.save("models/distilbert_model")

In [ ]:
#testing if predictor can loaded from storage
predictor = ktrain.load_predictor("models/distilbert_model")

In [ ]:
#testing again
import time 

message = 'I just broke up with my boyfriend'

start_time = time.time() 
prediction = predictor.predict(message)

print('predicted: {} ({:.2f})'.format(prediction, (time.time() - start_time)))

predicted: sadness (0.06)
